# Import Libraries

In [9]:
import numpy as np
import pandas as pd
import pickle

from sklearn import preprocessing

import mtcnn

import tensorflow as tf
from tensorflow import keras
from keras import Model, layers, Sequential, losses, optimizers

# Create Custom VGG16 Model

In [3]:
vgg_model = keras.applications.VGG19(include_top = False, input_shape = (48, 48, 3))
print(vgg_model.summary())

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

# Add fully-connected layers to the model

In [4]:
# emotions = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
n_class = 7

last_layer = vgg_model.get_layer("block5_pool").output
x = layers.Flatten()(last_layer)
x = layers.Dense(1024, activation = "relu", name = "fc1")(x)
x = layers.Dense(512, activation = "relu", name = "fc2")(x)
x = layers.Dense(256, activation = "relu", name = "fc3")(x)
x = layers.Dense(n_class, activation = "softmax", name = "output")(x)
custom_vgg_model = Model(vgg_model.input, x)
print(custom_vgg_model.summary())
custom_vgg_model.compile(
    loss = losses.CategoricalCrossentropy(from_logits = False),
    optimizer = tf.keras.optimizers.SGD(1e-3),
    metrics = ["accuracy"],
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

# Create Custom CNN Model

In [5]:
CNN_model = Sequential(
    layers = [
        layers.Input(shape = (48, 48, 3)),
        
        # Block 1
        layers.Conv2D(64, (3, 3), padding = "same", activation = "relu", name = "block1_conv1"),
        layers.Conv2D(64, (3, 3), padding = "same", activation = "relu", name = "block1_conv2"),
        layers.MaxPool2D((2, 2), strides=(2, 2), name = "block1_pool"),
        
        # Block 2
        layers.Conv2D(128, (3, 3), padding = "same", activation = "relu", name = "block2_conv1"),
        layers.Conv2D(128, (3, 3), padding = "same", activation = "relu", name = "block2_conv2"),
        layers.MaxPool2D((2, 2), strides=(2, 2), name = "block2_pool"),
        
        # Block 3
        layers.Conv2D(256, (3, 3), padding = "same", activation = "relu", name = "block3_conv1"),
        layers.Conv2D(256, (3, 3), padding = "same", activation = "relu", name = "block3_conv2"),
        layers.Conv2D(256, (3, 3), padding = "same", activation = "relu", name = "block3_conv3"),
        layers.Conv2D(256, (3, 3), padding = "same", activation = "relu", name = "block3_conv4"),
        layers.MaxPool2D((2, 2), strides=(2, 2), name = "block3_pool"),
        
        # Block 4
        layers.Conv2D(512, (3, 3), padding = "same", activation = "relu", name = "block4_conv1"),
        layers.Conv2D(512, (3, 3), padding = "same", activation = "relu", name = "block4_conv2"),
        layers.Conv2D(512, (3, 3), padding = "same", activation = "relu", name = "block4_conv3"),
        layers.Conv2D(512, (3, 3), padding = "same", activation = "relu", name = "block4_conv4"),
        layers.MaxPool2D((2, 2), strides=(2, 2), name = "block4_pool"),
        
        # Block 5
        layers.Conv2D(512, (3, 3), padding = "same", activation = "relu", name = "block5_conv1"),
        layers.Conv2D(512, (3, 3), padding = "same", activation = "relu", name = "block5_conv2"),
        layers.Conv2D(512, (3, 3), padding = "same", activation = "relu", name = "block5_conv3"),
        layers.Conv2D(512, (3, 3), padding = "same", activation = "relu", name = "block5_conv4"),
        layers.MaxPool2D((2, 2), strides=(2, 2), name = "block5_pool"),
        
        # Fully-connected layers for classification
        layers.Flatten(),
        layers.Dense(1024, activation = "relu", name = "fc1"),
        layers.Dense(512, activation = "relu", name = "fc2"),
        layers.Dense(256, activation = "relu", name = "fc3"),
        # 7 classes of emotion
        # ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
        layers.Dense(7, activation = "softmax", name = "classifier"),
    ],
    name = "Emotion_CNN"
)

print(CNN_model.summary())

Model: "Emotion_CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 12, 12, 256)       

In [13]:
CNN_model.compile(
    loss = losses.SparseCategoricalCrossentropy(from_logits = False),
    optimizer = keras.optimizers.Adam(learning_rate = 1e-3),
    metrics = ["accuracy"],
)

# Load Datasets

In [7]:
with open("../data/pickle/X_train.pkl", "rb") as fp:
    X_train = pickle.load(fp)
with open("../data/pickle/y_train.pkl", "rb") as fp:
    y_train = pickle.load(fp)

with open("../data/pickle/X_test.pkl", "rb") as fp:
    X_test= pickle.load(fp)
with open("../data/pickle/y_test.pkl", "rb") as fp:
    y_test = pickle.load(fp)

## Encode labels

In [11]:
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# Train the Custom CNN Model

In [14]:
CNN_model.fit(X_train, y_train, epochs = 20, batch_size = 64)

Epoch 1/20
  3/449 [..............................] - ETA: 59:15 - loss: 29.4354 - accuracy: 0.1927

KeyboardInterrupt: 